In [56]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import json
from collections import defaultdict
from datetime import datetime, timezone
from scipy import stats

In [45]:
# roughviz_metrics_path = "../metrics/roughViz_percentiles.csv"
# roughviz_metrics = pd.read_csv(roughviz_metrics_path)
# roughviz_metrics.iloc[:, 2:] = roughviz_metrics.iloc[:, 2:].apply(pd.to_numeric)

# twopasswords_metrics_path = "../metrics/twopasswords_percentiles.csv"
# twopasswords_metrics = pd.read_csv(twopasswords_metrics_path)
# twopasswords_metrics.iloc[:, 2:] = twopasswords_metrics.iloc[:, 2:].apply(pd.to_numeric)

# voicelistener_metrics_path = "../metrics/voice-listener_percentiles.csv"
# voicelistener_metrics = pd.read_csv(voicelistener_metrics_path)
# voicelistener_metrics.iloc[:, 2:] = voicelistener_metrics.iloc[:, 2:].apply(pd.to_numeric)


In [32]:
# Yujin's calculations for the Trustee label (MCPC)

roughviz_trustee_path = "../metrics/roughViz_prototype.csv"
roughviz_trustee = pd.read_csv(roughviz_trustee_path)
roughviz_trustee.iloc[:, 2:] = roughviz_trustee.iloc[:, 2:].apply(pd.to_numeric)

twopasswords_trustee_path = "../metrics/twopasswords_prototype.csv"
twopasswords_trustee = pd.read_csv(twopasswords_trustee_path)
twopasswords_trustee.iloc[:, 2:] = twopasswords_trustee.iloc[:, 2:].apply(pd.to_numeric)

voicelistener_trustee_path = "../metrics/voice-listener_prototype.csv"
voicelistener_trustee = pd.read_csv(voicelistener_trustee_path)
voicelistener_trustee.iloc[:, 2:] = voicelistener_trustee.iloc[:, 2:].apply(pd.to_numeric)


In [43]:
repos_trustee = {
    "roughViz": roughviz_trustee,
    "twopasswords": twopasswords_trustee,
    "voice_listener": voicelistener_trustee,
}

In [63]:
# # Grades based on 20%ile
# balanced_grades = {
#     "F": [0, 19],
#     "D": [20, 39],
#     "C": [40, 59],
#     "B": [60, 79],
#     "A": [80, 100],
# }

grades_bins = [0, 200, 400, 600, 800, 1010]
grades_labels = ["F", "D", "C", "B", "A"]

In [64]:
z_grades_bins = [-100, -1.5, -0.5, 0.5, 1.5, 100]

# Calculate Trust Component Label Metrics

In [74]:
for repo in repos_trustee:
    component = repos_trustee[repo].fillna(0)
    component["community_activity_and_integrity"] = component[["popularity", "code_contribution", "contributor_participation", "contributor_growth"]].mean(axis=1)
    component["maintenance_and_goodwill"] = component[["issues_maintenance", "community_documentation", "code_maintenance", "maintainer_history"]].mean(axis=1)
    component["code_quality"] = component[["dependencies_health", "review_coverage", "testing_quality", "project_maturity"]].mean(axis=1)
    component["component_avg"] = component[["community_activity_and_integrity", "maintenance_and_goodwill", "code_quality"]].mean(axis=1)
    
    component["component_rank"] = component["component_avg"].rank()
    component["component_rank_grade"] = pd.cut(component["component_rank"], bins=grades_bins, right=False, labels=grades_labels)

    component["trustee_rank"] = component["avg_percentile"].rank()
    component["trustee_rank_grade"] = pd.cut(component["trustee_rank"], bins=grades_bins, right=False, labels=grades_labels)
    
    component["component_zscore"] = stats.zscore(component["component_avg"])
    component["component_z_grade"] = pd.cut(component["component_zscore"], bins=z_grades_bins, right=False, labels=grades_labels)

    component["trustee_zscore"] = stats.zscore(component["avg_percentile"])
    component["trustee_z_grade"] = pd.cut(component["trustee_zscore"], bins=z_grades_bins, right=False, labels=grades_labels)

    output_file = "../metrics/" + repo + "_both_prototypes_v2.csv"
    component.to_csv(output_file, index=False)


^the calculation for Popularity may be incorrect (avg of avg's, instead of ground-up calculation), but it should be close enough and saves some dev time - unless yujin has the code and can quickly re-calculate, this should be fine as-is